In [ ]:
!pip install transformers datasets evaluate bitsandbytes pinecone-client pypdf langchain langchain-community langchain-pinecone -qU pinecone-notebooks

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 110.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 89.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
!pip install -U bitsandbytes

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import time
import warnings
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch
from langchain_pinecone import PineconeEmbeddings
from langchain.vectorstores import Pinecone as PineconeStore
import pinecone
import os
import json
from pinecone import Pinecone
from langchain import HuggingFacePipeline
warnings.filterwarnings("ignore")

In [ ]:
shaps_path = "/content/drive/MyDrive/Capstone/data/test_data_shaps.csv"
test_data_path = "/content/drive/MyDrive/Capstone/data/test_data_transformed.csv"

In [ ]:
shaps = pd.read_csv(shaps_path)
shaps.head()

,loan_amnt,int_rate,installment,sub_grade,emp_length,annual_inc,dti,open_acc,pub_rec,revol_bal,...,addr_state_WA,addr_state_WI,addr_state_WV,addr_state_WY,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,Prediction Proba,Target
0,0.151707,-0.309311,-0.016923,-0.005849,0.010770,-0.147776,-0.085177,0.174910,0.007480,-0.064157,...,-0.003841,-0.001954,-0.001211,-0.000332,0.024905,0.0,-0.012774,0.0,0.531337,1
1,0.162436,-0.276263,0.110912,-0.172780,-0.038879,0.062768,0.008214,0.169840,0.013174,-0.099606,...,0.198400,-0.000101,-0.000867,-0.000537,0.036792,0.0,-0.007685,0.0,0.677305,1
2,-0.088292,-0.874404,-0.051398,0.226191,0.038157,0.163688,0.015543,0.011530,0.015154,0.241538,...,-0.007974,-0.000437,-0.001895,-0.000340,0.018119,0.0,-0.013604,0.0,0.439310,0
3,-0.044761,0.019172,-0.033732,0.417962,-0.008443,0.029195,-0.016255,0.144786,-0.020869,-0.316588,...,-0.006821,0.000080,-0.001032,-0.000106,0.087271,0.0,-0.017433,0.0,0.812301,1
4,0.228717,-0.246398,0.184627,0.015084,0.061386,0.017530,0.051524,0.028956,0.012954,-0.039837,...,-0.005104,-0.001196,-0.001736,-0.000581,0.031900,0.0,-0.007234,0.0,0.814199,1


In [ ]:
df = pd.read_csv(test_data_path)
df.head()

,loan_amnt,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,purpose,...,revol_bal,revol_util,total_acc,initial_list_status,application_type,mort_acc,pub_rec_bankruptcies,term_months,earliest_cr_line_year,earliest_cr_line_month
0,5000.0,13.99,170.87,C,3,4.0,RENT,22680.0,Source Verified,vacation,...,4200.0,29.2,14.0,w,Individual,0.0,0.0,36,1994,8
1,4375.0,15.99,153.80,C,5,0.5,RENT,70000.0,Source Verified,credit_card,...,2638.0,61.3,8.0,w,Individual,0.0,0.0,36,1996,12
2,35000.0,19.99,927.10,E,1,10.0,MORTGAGE,141555.0,Verified,debt_consolidation,...,36900.0,101.1,31.0,w,Individual,4.0,0.0,60,1997,9
3,15000.0,18.94,388.62,D,2,3.0,MORTGAGE,72000.0,Verified,debt_consolidation,...,206.0,6.2,15.0,w,Individual,2.0,1.0,60,2003,1
4,3025.0,19.42,111.53,D,3,10.0,MORTGAGE,64655.0,Source Verified,debt_consolidation,...,5913.0,85.7,12.0,w,Individual,1.0,0.0,36,2002,8


In [ ]:
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "Virgin Islands, U.S.": "VI",
}

# invert the dictionary
abbrev_to_us_state = dict(map(reversed, us_state_to_abbrev.items()))

In [ ]:
data_dict = {"loan_amnt": "Loan amount applied for by borrower",
             "int_rate": "Estimated interest rate on loan",
             "installment": "Monthly installments owed",
             "grade": "Estimated loan grade",
             "sub_grade": "Estimated loan sub-grade",
             "emp_length": "Employment length",
             "home_ownership": "Home ownership status",
             "annual_inc": "Annual income of borrower",
             "verification_status": "Income verification status",
             "purpose": "purpose of loan provided by borrower",
             "title": "Loan title provided by the borrower",
             "addr_state": "Residence state of borrower",
             "dti": "Monthly debt-to-income ratio",
             "open_acc": "Number of borrower's open credit lines",
             "pub_rec": "Number of derogatory public records",
             "revol_bal": "Total credit revolving balance",
             "revol_util": "Revolving line utilization rate",
             "total_acc": "Total number of borrower's credit lines",
             "initial_list_status": "Initial listing status of loan",
             "application_type": "Individual or joint application",
             "mort_acc": "Number of mortgage accounts",
             "pub_rec_bankruptcies": "Number of public record bankruptices",
             "term_months": "Number of monthly payments in loan",
             "earliest_cr_line_year": "Year earliest credit line was opened",
             "earliest_cr_line_month": "Month earliest credit line was opened"
            }

### Download tokenizer and model

In [ ]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False)

In [ ]:
base_model = "meta-llama/Llama-3.2-3B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map={"": 0},
    use_auth_token='auth_token'
)

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    base_model,
    trust_remote_code=True,
    token="auth_token",
)

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [ ]:
def preprocess_function(prompt,tokenizer = tokenizer):
    return tokenizer(prompt)

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
# tokenizer.padding_side = "right"

# Prompt construction - helper functions

In [ ]:
def get_pred(idx = 0):
    shap_vals = shaps.iloc[idx,:].copy()
    proba = shap_vals['Prediction Proba']

    return proba

In [ ]:
def get_features_shaps(idx = 0):
  data = df.iloc[idx,:].copy().to_frame().T
  shap_vals = shaps.iloc[idx,:].copy().to_frame().T
  shap_vals.drop(columns = ["Prediction Proba", "Target"], inplace = True)

  feature_shap_vals = []

  for col in data:
    if col in shap_vals:
      feature_shap_vals.append(f"feature_name: {col}, feature_value: {data[col].values[0]}, shap_value: {shap_vals[col].values[0]}\n")
    else:
      try:
        col_val = data[col].values[0]
        shap_val = shap_vals[f"{col}_{col_val}"].values[0]
        if col == "addr_state":
          col_val = abbrev_to_us_state.get(col_val)
        feature_shap_vals.append(f"feature_name: {col}, feature_value: {col_val}, shap_value: {shap_val}\n")
      except Exception as e:
        raise e

  return "\n".join(feature_shap_vals)

In [ ]:
def get_addr_state(idx = 0):
  data = df.iloc[idx,:].copy().to_frame().T
  addr_state = col_val = abbrev_to_us_state.get(data["addr_state"].values[0])
  return addr_state

# Pinecone

In [ ]:
pinecone_api = "pinecone_token"

In [ ]:
pc = Pinecone(api_key=pinecone_api)

In [ ]:
try:
    indexes = pc.list_indexes()
    print("Successfully connected to Pinecone.")
    print("Existing indexes:", indexes)
except Exception as e:
    print(f"Failed to connect to Pinecone: {e}")

Successfully connected to Pinecone.
Existing indexes: [{
    "name": "coral-x",
    "dimension": 1024,
    "metric": "cosine",
    "host": "coral-x-w1xpav1.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "deletion_protection": "disabled"
}, {
    "name": "rag-test",
    "dimension": 1024,
    "metric": "cosine",
    "host": "rag-test-w1xpav1.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "deletion_protection": "disabled"
}]


In [ ]:
index_name = "coral-x"
host = pc.describe_index(index_name)["host"]
host

'coral-x-w1xpav1.svc.aped-4627-b74a.pinecone.io'

In [ ]:
index = pc.Index(host = host)

In [ ]:
index.describe_index_stats()

{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {'data-context-docs': {'vector_count': 18},
                'global-context-docs': {'vector_count': 258}},
 'total_vector_count': 276}

In [ ]:
model_name = 'multilingual-e5-large'
embeddings = PineconeEmbeddings(
    model=model_name,
    pinecone_api_key=pinecone_api)

In [ ]:
vectordb = PineconeStore(index, embeddings, text_key = "text")

# Custom Pipeline - Final

In [ ]:
def rm_duplicate_lines(text, sep = "\n"):
  lines = text.split(sep)
  lines_set = set()
  clean_text = ""
  for line in lines:
    if line not in lines_set:
      # clean_text += "\n".join(line)
      clean_text += line + "\n"
      lines_set.add(line)

  return clean_text

In [ ]:
def format_docs(results, clean = False):
  rel_docs = results["matches"]
  full_doc = ""
  for doc in rel_docs:
    full_doc += "".join(doc["metadata"]["text"])
    full_doc += "\n"

  if clean == True:
    clean_docs = rm_duplicate_lines(text = full_doc)
    return clean_docs

  return full_doc

In [ ]:
def retrieve(query, namespace, embeddings = embeddings, top_k = 3, pc_index = index, clean = False):
  results = index.query(
      namespace = namespace,
      vector = embeddings.embed_query(query),
      top_k = top_k,
      include_metadata = True,
      include_values = False
  )

  return format_docs(results, clean = clean)

In [ ]:
def generate_context(idx = 0):
  context = retrieve(
      query = get_features_shaps(idx = idx),
      namespace = "data-context-docs",
      top_k = 5,
      clean = True)

  state = get_addr_state(idx = idx)

  context += retrieve(
      query = f"What are the statistics for the state of {state}",
      namespace = "data-context-docs",
      top_k = 1,
      clean = True)

  doc_set = set()
  for key,val in data_dict.items():
    try:
      doc = retrieve(
          query = f"How does {val} affect loan eligibility?",
          namespace = "global-context-docs",
          top_k = 1,
          clean = True
      )

      if doc not in doc_set:
        context += doc
        doc_set.add(doc)

    except Exception as e:
        raise e

  return context

In [ ]:
# instructions - If the feature is loan amount, dti or annual income, compare it's value to the respective median value for the state of {addr_state}. \
# Do not compare this information for any other features than those mentioned.
# output - If the feature is loan amount, dti or annual income - the comparison of the borrower's value of that feature against the feature's median value for the state of {addr_state}, \
# and how it influences the loan repayment probability specifically, for the borrower. Make sure the comparison is accurate and correct. \
# Do not answer if the feature is other than the ones mentioned.

def custom_prompt(context, idx = 0):

  ## Task Description
  system_prompt = f"""You are an assistant for a loan agent who is analyzing a borrower's loan application, \
The agent has used an XGBoost machine learning model to predict probability of that borrower paying back the loan timely.
Your task is to analyze as per the given instructions - the borrower features provided to the model, the model's prediction and the SHAP values generated by the model; and \
provide a coherent and non-technical explanation on how the provided features influence the model's prediction.\n"""

  pred_features_intro = """\nPrediction, features and SHAP values input:\n"""
  pred_string = f"""- Prediction for the instance: {round(get_pred(idx = idx) * 100, 2)}% probability of paying back the loan timely.\n"""
  features_string = f"""- Features values and SHAP values of the instance:\n{get_features_shaps(idx = idx)}\n"""
  addr_state = get_addr_state(idx = idx)

  instructions = f"""Instructions:
Rank the features based on the magnitude of SHAP values from highest to lowest, regardless of the direction, and pick the top 10 features.

For each feature in the top 10 features:
- Retrieve the feature definition for the feature.
- Utilize the definition to find how the feature influences loan eligibility.
- Utilize the feature's provided SHAP value to determine it's importance for decision making \
Make sure to interpret the magnitude and direction of the SHAP value correctly.
- Determine how the feature and it's value contribute to the prediction, such as whether a higher value increases the loan repayment probability or decreases it.\
Make sure you are considering the general context as well as the context of the particular borrower.

Output format:
Iterate over each feature in the top 10 features, and output the following sections for each:
1) Feature Definition:
- The title of the feature
- The value of this feature for the borrower
- A brief explanation of what this feature represents, and how it influences the loan repayment probability generally as well as specifically for the borrower.
- Is there a median value for this feature available for the state of {addr_state}? \
If yes, how does the borrower's value for this feature compare to the state median for this feature? If no median value is available then do note compare.
The length of this section should be between 2 to 4 sentences.
2) SHAP Context:
- The SHAP value for the feature
- A brief explanation of the feature's influence on the model's prediction for this particular borrower as represented by the corresponding SHAP value.
Make sure to interpret the magnitude and direction of the SHAP value correctly. The length of this section should be between 1 to 3 sentences.

Conclude with a brief summary of how the top 10 influencing features affect the decision and contribute to the model's prediction. This summary should be under 5 sentences.\n"""


  context = f"""Context:\n{context}\nOutput:\n"""

  prompt = system_prompt + instructions + pred_features_intro + pred_string + features_string + context
  return prompt

In [ ]:
temperature = 0.35
max_length = 6000

In [ ]:
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=max_length, truncation = True)

Device set to use cuda:0


In [ ]:
llm_agent = HuggingFacePipeline(
    pipeline=pipe,
    model_kwargs={"temperature": temperature, "max_length": max_length},
)

## RAG results

In [ ]:
def rag_pipeline(idx = 0):
  start = time.perf_counter()
  context = generate_context(idx = idx)
  prompt = custom_prompt(context = context, idx = idx)
  results = llm_agent.invoke(prompt)
  end = time.perf_counter()
  elapsed_time = round(end-start,2)
  return results, elapsed_time

In [ ]:
def rag_generate(idx = 0):
  result, elapsed_time = rag_pipeline(idx = idx)
  result_index = result.find("Output:\n")
  filter_result = result[result_index:]
  return result, filter_result, elapsed_time

In [ ]:
def evaluate(rag = True):
  samples = np.arange(50,1001,50)
  results = []

  if rag:
    print("Generating resuts with RAG")
    for sample in samples:
      full_result, output, elapsed_time = rag_generate(idx = sample)
      results.append({"sample_index": int(sample),
                      "model_output": output,
                      "full_output": full_result,
                      "rag": 1,
                      "pipeline_time": elapsed_time})
      print(f"Results logged for index {sample} with pipeline_time of {elapsed_time} seconds")

  else:
    print("Generating results without retrieval")
    for sample in samples:
      full_result, output, elapsed_time = no_rag_generate(idx = sample)
      results.append({"sample_index": int(sample),
                      "model_output": output,
                      "full_output": full_result,
                      "rag": 0,
                      "pipeline_time": elapsed_time})
      print(f"Results logged for index {sample} with pipeline_time of {elapsed_time} seconds")

  return results

In [ ]:
rag_results = evaluate(rag = True)

Generating resuts with RAG
Results logged for index 50 with pipeline_time of 84.58 seconds
Results logged for index 100 with pipeline_time of 88.08 seconds
Results logged for index 150 with pipeline_time of 81.94 seconds
Results logged for index 200 with pipeline_time of 81.47 seconds
Results logged for index 250 with pipeline_time of 87.16 seconds
Results logged for index 300 with pipeline_time of 91.98 seconds
Results logged for index 350 with pipeline_time of 80.25 seconds


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Results logged for index 400 with pipeline_time of 66.28 seconds
Results logged for index 450 with pipeline_time of 79.96 seconds
Results logged for index 500 with pipeline_time of 65.81 seconds
Results logged for index 550 with pipeline_time of 81.47 seconds
Results logged for index 600 with pipeline_time of 75.63 seconds
Results logged for index 650 with pipeline_time of 83.86 seconds
Results logged for index 700 with pipeline_time of 85.49 seconds
Results logged for index 750 with pipeline_time of 85.12 seconds
Results logged for index 800 with pipeline_time of 77.22 seconds
Results logged for index 850 with pipeline_time of 68.79 seconds
Results logged for index 900 with pipeline_time of 83.86 seconds
Results logged for index 950 with pipeline_time of 84.37 seconds
Results logged for index 1000 with pipeline_time of 78.77 seconds


In [ ]:
for item in rag_results:
  item['sample_index'] = int(item['sample_index'])

In [ ]:
eval_path = "/content/drive/MyDrive/Capstone/eval_samples"

In [ ]:
with open(os.path.join(eval_path, "rag_outputs.json"), "w") as f:
  json.dump(rag_results, f, indent = 4)

## No RAG results

In [ ]:
def generate_pipeline(idx = 0):
  start = time.perf_counter()
  prompt = custom_prompt(context = "", idx = idx)
  results = llm_agent.invoke(prompt)
  end = time.perf_counter()
  elapsed_time = round(end-start,2)
  return results, elapsed_time

In [ ]:
def no_rag_generate(idx = 0):
  result, elapsed_time = generate_pipeline(idx = idx)
  result_index = result.find("Output:\n")
  filter_result = result[result_index:]
  return result, filter_result, elapsed_time

In [ ]:
no_rag_results = evaluate(rag = False)

Generating results without retrieval
Results logged for index 50 with pipeline_time of 105.74 seconds
Results logged for index 100 with pipeline_time of 194.38 seconds
Results logged for index 150 with pipeline_time of 66.48 seconds
Results logged for index 200 with pipeline_time of 194.17 seconds
Results logged for index 250 with pipeline_time of 70.12 seconds
Results logged for index 300 with pipeline_time of 45.75 seconds
Results logged for index 350 with pipeline_time of 194.11 seconds
Results logged for index 400 with pipeline_time of 114.85 seconds
Results logged for index 450 with pipeline_time of 57.89 seconds
Results logged for index 500 with pipeline_time of 64.74 seconds
Results logged for index 550 with pipeline_time of 102.58 seconds
Results logged for index 600 with pipeline_time of 77.35 seconds
Results logged for index 650 with pipeline_time of 193.08 seconds
Results logged for index 700 with pipeline_time of 192.7 seconds
Results logged for index 750 with pipeline_time

In [ ]:
with open(os.path.join(eval_path, "no_rag_outputs.json"), "w") as f:
  json.dump(no_rag_results, f, indent = 4)